めちゃくちゃ見づらいコードでごめんなさい

for で1行ずつ見てたりするのでゴミです。

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tqdm import tqdm
from sklearn.calibration import CalibratedClassifierCV
from IPython.display import display
import pickle
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, KFold
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import PolynomialFeatures
from IPython.display import HTML

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
HTML("""
<button id="code-show-switch-btn">スクリプトを非表示にする</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("スクリプトを表示する");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("スクリプトを非表示にする");
    }
    }

$("#code-show-switch-btn").click(switch_display_setting);
</script>
""")

In [3]:
def func(x1, x2, effect='add'):
    if effect == 'add':
        res = x1 + x2
    elif effect == 'sub':
        res = x1 - x2
    elif effect == 'multi':
        res = x1 * x2
    elif effect == 'div':
        res = x1 / x2
    return res

def cross_calc_matrix(df, cols, addition=True, subtraction=True, multiple=True, division=True):
    # それぞれのポケモンで、colsから2つの能力値を足したり引いたり掛けたり割ったりする
    
    n_cols = len(cols)

    df_for_obj = df[cols]
    ar_for_obj = df_for_obj.values

    actions = []
    if addition:
        actions.append('add')
    if subtraction:
        actions.append('sub')
    if multiple:
        actions.append('multi')
    if division:
        actions.append('div')
    result = np.zeros([df.shape[0], int(len(actions) * n_cols * (n_cols - 1) / 2)])
    cnt = 0
    new_cols = []
    for act in actions:
        for ix_col1 in range(n_cols):
            for ix_col2 in range(ix_col1 + 1, n_cols):
                result[:, cnt] = func(ar_for_obj[:, ix_col1], ar_for_obj[:, ix_col2], act)
                cnt += 1
                new_cols.append(cols[ix_col1] + '_' + act + '_' + cols[ix_col2])
    df_result  = pd.DataFrame(result, columns=new_cols)
    return df_result

def make_matrix(df_game, df_status, X_cols, training=False):
    game_ar = np.array(df_game)
    status_ar = np.array(df_status)
    
    # 対戦する二つのポケモンの能力を並べたものをresult_arに入れる
    result_ar = np.zeros([len(df_game), len(X_cols)*2])
    y_label = np.zeros(len(df_game))# First pokemonが勝てば1
    
    X_cols1 = [i + '_pok1' for i in X_cols] # 能力値のcolumnがぶつからないようにpok1, pok2で区別
    X_cols2 = [i + '_pok2' for i in X_cols]
    X_cols_for_matrix = X_cols1 + X_cols2
    
    for i in tqdm(range(len(df_game))):
        first_pok_id = game_ar[i, 0]
        second_pok_id = game_ar[i, 1]
        winner_id = game_ar[i, 2]
        X_tmp1 = status_ar[first_pok_id-1, 4:] # -1はpokemon# が1始まりの為
        X_tmp2 = status_ar[second_pok_id-1, 4:]
        
        X_tmp = np.hstack([X_tmp1.reshape(1, -1), X_tmp2.reshape(1, -1)])
        result_ar[i] = X_tmp
        if first_pok_id==winner_id:
            y_label[i] = 1
    df_res = pd.DataFrame(result_ar, columns=X_cols_for_matrix)
    df_res['target'] = y_label
    return df_res

def relative_matrix(df_mx, obj_cols = 'auto'):
    if obj_cols == 'auto':
        default_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Speed', 'Sp. Def', 'Generation', 'Legendary']
        continuos_cols_1 = [i + '_pok1' for i in default_cols]
        continuos_cols_2 = [i + '_pok2' for i in default_cols]
    else:
        continuos_cols_1 = [i + '_pok1' for i in obj_cols]
        continuos_cols_2 = [i + '_pok2' for i in obj_cols]
    
    df_res = pd.DataFrame()
    for i, col in enumerate(continuos_cols_1):
        new_col = col[:-1]+'_delta'
        df_res[new_col] = df_mx[continuos_cols_1[i]].values - df_mx[continuos_cols_2[i]].values
#         new_col = col[:-1]+'_sum'
#         df_res[new_col] = df_mx[continuos_cols_1[i]].values + df_mx[continuos_cols_2[i]].values
#         new_col = col[:-1]+'_product'
#         df_res[new_col] = df_mx[continuos_cols_1[i]].values * df_mx[continuos_cols_2[i]].values
    return df_res

def make_matrix_test(df_game, df_status, X_cols, training=False):
    game_ar = np.array(df_game)
    status_ar = np.array(df_status)
    
    # 対戦する二つのポケモンの能力を並べたものをresult_arに入れる
    result_ar = np.zeros([len(df_game), len(X_cols)*2])
    y_label = np.zeros(len(df_game))# First pokemonが勝てば1
    
    X_cols1 = [i + '_pok1' for i in X_cols] # 能力値のcolumnがぶつからないようにpok1, pok2で区別
    X_cols2 = [i + '_pok2' for i in X_cols]
    X_cols_for_matrix = X_cols1 + X_cols2
    
    for i in tqdm(range(len(df_game))):
        first_pok_id = game_ar[i, 0]
        second_pok_id = game_ar[i, 1]
        X_tmp1 = status_ar[first_pok_id-1, 4:] # -1はpokemon# が1始まりの為
        X_tmp2 = status_ar[second_pok_id-1, 4:]
        
        X_tmp = np.hstack([X_tmp1.reshape(1, -1), X_tmp2.reshape(1, -1)])
        result_ar[i] = X_tmp
    df_res = pd.DataFrame(result_ar, columns=X_cols_for_matrix)
    return df_res

def relative_matrix_test(df_mx, obj_cols = 'auto'):
    if obj_cols == 'auto':
        default_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Speed', 'Sp. Def', 'Generation', 'Legendary']
        continuos_cols_1 = [i + '_pok1' for i in default_cols]
        continuos_cols_2 = [i + '_pok2' for i in default_cols]
    else:
        continuos_cols_1 = [i + '_pok1' for i in obj_cols]
        continuos_cols_2 = [i + '_pok2' for i in obj_cols]
    
    df_res = pd.DataFrame()
    for i, col in enumerate(continuos_cols_1):
        new_col = col[:-1]+'_delta'
        df_res[new_col] = df_mx[continuos_cols_1[i]].values - df_mx[continuos_cols_2[i]].values
    return df_res

def calc_magni(type1_1st, type2_1st, type1_2nd, type2_2nd):
    # ポケモンタイプに従って攻撃倍率計算
    if not np.logical_or(type(type1_1st) == float, type(type1_2nd) == float):
        attack_maguni_1st = df_chart.at[type1_1st, type1_2nd]
        if type(type2_2nd) == str:
            attack_maguni_1st *= df_chart.at[type1_1st, type2_2nd]
            if type(type2_1st) == str:
                attack_maguni_1st *= df_chart.at[type2_1st, type1_2nd]
                attack_maguni_1st *= df_chart.at[type2_1st, type2_2nd]
        else:
            if type(type2_1st) == str:
                attack_maguni_1st *= df_chart.at[type2_1st, type1_2nd]

        attack_maguni_2nd = df_chart.at[type1_2nd, type1_1st]
        if type(type2_1st) == str:
            attack_maguni_2nd *= df_chart.at[type1_2nd, type2_1st]
            if type(type2_2nd) == str:
                attack_maguni_2nd *= df_chart.at[type2_2nd, type1_1st]
                attack_maguni_2nd *= df_chart.at[type2_2nd, type2_1st]
        else:
            if type(type2_2nd) == str:
                attack_maguni_2nd *= df_chart.at[type2_2nd, type1_1st]
    else:
        attack_maguni_1st = 1
        attack_maguni_2nd = 1
    return attack_maguni_1st, attack_maguni_2nd
def WoE(df, test, col, target_col):
    pos_outcom = df[target_col].sum()
    neg_outcom = len(df) - pos_outcom
    col_unique = df[col].unique()
    res = pd.DataFrame()
    res[col] = col_unique
    res['po'] = 0
    res['ne'] = 0
    a = df.groupby([col, target_col]).count()['HP_pok1']
    for i in tqdm(col_unique):
        try:
            res.loc[res[col] == i, 'po'] = a[i][1]
        except:
            res.loc[res[col] == i, 'po'] = 0
        try:
            res.loc[res[col] == i, 'ne'] = a[i][0]
        except:
            res.loc[res[col] == i, 'ne'] = 0
    res['woe_po'] = (res['po']+0.5) / pos_outcom
    res['woe_ne'] = (res['ne']+0.5) / neg_outcom
    res_col = col + 'woe'
    res[res_col] = np.log(res['woe_po']/res['woe_ne'])
    res1 = pd.merge(df, res[[col, res_col]], on=col)
    res2 = pd.merge(test, res[[col, res_col]], on=col, how='left')
    return res1, res2

dataの読み込みとGenerationとTypeをOHE(one hot encoding)

(Typeは順序関係ないっぽいので、Two-hot-encodingになってる)

In [4]:
# dataのinport
df_tr = pd.read_csv('../input/train.csv', index_col='id')
df_ts = pd.read_csv('../input/test.csv', index_col='id')
df_pok = pd.read_csv('../input/pokemon.csv')
df_chart = pd.read_csv('../input/type.csv', index_col=0)

# True, Falseを1,0に置き換える
df_pok.loc[df_pok['Legendary'], 'Legendary'] = 1
df_pok.loc[df_pok['Legendary']==False, 'Legendary'] = 0
display(df_pok.head())

# TypeをOHE(one-hot-encoding)
label_encoder_type = LabelEncoder()
ohe_type = OneHotEncoder()
Type_names = df_chart.index.tolist()
Type_names.append('nan')
label_encoder_type.fit(Type_names)
type1_ohe = ohe_type.fit_transform(label_encoder_type.transform(list(df_pok['Type 1'])).reshape(-1, 1)).todense()
type2_ohe = ohe_type.transform(label_encoder_type.transform(list(df_pok['Type 2'])).reshape(-1, 1)).todense()
df_type_ohe = pd.DataFrame(type1_ohe + type2_ohe, columns=label_encoder_type.classes_) # Type1, Type2に優劣はないらしい

# GenerationをOHE
label_encoder_gene = LabelEncoder()
ohe_gene = OneHotEncoder()
generation_names = ['gene_' + str(i + 1) for i in range(df_pok['Generation'].nunique())]
generation_ohe = ohe_gene.fit_transform(label_encoder_gene.fit_transform(list(df_pok['Generation'])).reshape(-1, 1)).todense()
df_generation_ohe = pd.DataFrame(generation_ohe, columns=generation_names)

# TypeとGenerationのOHEをdf_pokに連結
df_pok = pd.concat([df_pok, df_type_ohe, df_generation_ohe], axis=1)
Cols = df_pok.columns.drop(['#','Name'])
display(df_type_ohe.head())
display(df_generation_ohe.head())

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,0
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,0
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,0
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,0
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,0


,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


trainの対戦表に従って、1st pokemon から 2nd pokemonとの攻撃倍率を計算。それぞれの交差項も計算

In [5]:
df_mx = df_tr.copy()
type1_1st_list = []
type2_1st_list = []
type1_2nd_list = []
type2_2nd_list = []
attack_1st_magni_list =[]
attack_2nd_magni_list =[]

# 対戦表を1行ずつ見ていく
for i in tqdm(range(len(df_mx))):
    # First pokemonの番号をとってくる
    pok_1st = df_mx.loc[i, 'First_pokemon']
    # First pokemonのtypeをとってくる
    type1_1st = df_pok.loc[pok_1st-1, 'Type 1']
    type2_1st = df_pok.loc[pok_1st-1, 'Type 2']    
    type1_1st_list.append(type1_1st)
    type2_1st_list.append(type2_1st)
    
    # Second pokemonに対しても同じ操作
    pok_2nd = df_mx.loc[i, 'Second_pokemon']
    type1_2nd = df_pok.loc[pok_2nd-1, 'Type 1']
    type2_2nd = df_pok.loc[pok_2nd-1, 'Type 2']
    type1_2nd_list.append(type1_2nd)
    type2_2nd_list.append(type2_2nd)
    
    # 対戦pokemonのTypeに基づいて
    attack_1st_magni, attack_2nd_magni = calc_magni(type1_1st, type2_1st, type1_2nd, type2_2nd)
    attack_1st_magni_list.append(attack_1st_magni)
    attack_2nd_magni_list.append(attack_2nd_magni)
df_mx['1st_type1'] = type1_1st_list
df_mx['1st_type2'] = type2_1st_list
df_mx['2nd_type1'] = type1_2nd_list
df_mx['2nd_type2'] = type2_2nd_list
df_mx['attack_1st_maguni'] = attack_1st_magni_list
df_mx['attack_2nd_maguni'] = attack_2nd_magni_list
df_tr_maguni = df_mx[['attack_1st_maguni', 'attack_2nd_maguni']].copy()
cross_maguni_tr = cross_calc_matrix(df_tr_maguni, ['attack_1st_maguni', 'attack_2nd_maguni'], division=False)
df_tr_maguni = pd.concat([df_tr_maguni, cross_maguni_tr], axis=1)
df_tr_maguni.head()

100%|██████████████████████████████████████████████████████████████████████████| 40000/40000 [00:08<00:00, 4498.17it/s]


,attack_1st_maguni,attack_2nd_maguni,attack_1st_maguni_add_attack_2nd_maguni,attack_1st_maguni_sub_attack_2nd_maguni,attack_1st_maguni_multi_attack_2nd_maguni
id,,,,,
0,2.0,0.5,2.5,1.5,1.0
1,0.5,1.0,1.5,-0.5,0.5
2,8.0,0.5,8.5,7.5,4.0
3,2.0,1.0,3.0,1.0,2.0
4,1.0,1.0,2.0,0.0,1.0


testの対戦表に従って↑と同じ操作

In [6]:
# 上記のものをtestdataにもやります
df_mx = df_ts.copy()
type1_1st_list = []
type2_1st_list = []
type1_2nd_list = []
type2_2nd_list = []
attack_1st_magni_list =[]
attack_2nd_magni_list =[]
for i in tqdm(range(len(df_mx))):
    pok_1st = df_mx.loc[i, 'First_pokemon']
    type1_1st = df_pok.loc[pok_1st-1, 'Type 1']
    type2_1st = df_pok.loc[pok_1st-1, 'Type 2']    
    type1_1st_list.append(type1_1st)
    type2_1st_list.append(type2_1st)
    
    pok_2nd = df_mx.loc[i, 'Second_pokemon']
    type1_2nd = df_pok.loc[pok_2nd-1, 'Type 1']
    type2_2nd = df_pok.loc[pok_2nd-1, 'Type 2']
    type1_2nd_list.append(type1_2nd)
    type2_2nd_list.append(type2_2nd)
#     print(type1_1st, type2_1st, type1_2nd, type2_2nd)
    attack_1st_magni, attack_2nd_magni = calc_magni(type1_1st, type2_1st, type1_2nd, type2_2nd)
    attack_1st_magni_list.append(attack_1st_magni)
    attack_2nd_magni_list.append(attack_2nd_magni)
df_mx['1st_type1'] = type1_1st_list
df_mx['1st_type2'] = type2_1st_list
df_mx['2nd_type1'] = type1_2nd_list
df_mx['2nd_type2'] = type2_2nd_list
df_mx['attack_1st_maguni'] = attack_1st_magni_list
df_mx['attack_2nd_maguni'] = attack_2nd_magni_list
df_ts_maguni = df_mx[['attack_1st_maguni', 'attack_2nd_maguni']].copy()
cross_maguni_ts = cross_calc_matrix(df_ts_maguni, ['attack_1st_maguni', 'attack_2nd_maguni'], division=False)
df_ts_maguni = pd.concat([df_ts_maguni, cross_maguni_ts], axis=1)
df_ts_maguni.head()

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4920.98it/s]


,attack_1st_maguni,attack_2nd_maguni,attack_1st_maguni_add_attack_2nd_maguni,attack_1st_maguni_sub_attack_2nd_maguni,attack_1st_maguni_multi_attack_2nd_maguni
id,,,,,
0,4.0,0.25,4.25,3.75,1.0
1,1.0,1.00,2.00,0.00,1.0
2,1.0,1.00,2.00,0.00,1.0
3,1.0,1.00,2.00,0.00,1.0
4,2.0,1.00,3.00,1.00,2.0


たぶん意味ない↓

In [7]:
# 対戦に関係のないType相性を変数にした
# たぶん意味ない

# pokemonが持ってるTypeの他のTypeに対する攻撃倍率とか増やしてみる
attack_cols = ['attack_on_' + i for i in df_chart.columns]
attack_matrix = np.ones([800, len(attack_cols)])
for i in tqdm(range(800)):
    type1 = df_pok.loc[i, 'Type 1']
    if type1 in Type_names:
        attack_matrix[i, :] *= df_chart.loc[type1, :].values
    
    type2 = df_pok.loc[i, 'Type 2']
    if type2 in Type_names:
        attack_matrix[i, :] *= df_chart.loc[type2, :].values
df_pok = pd.concat([df_pok, pd.DataFrame(attack_matrix, columns=attack_cols)], axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 1826.38it/s]


自分と相手の種族値に関して交差項をとる

例えば攻撃順序が決まる速さの差は重要だなと思っていたので、そのほかの似た変数も一緒に作った

In [8]:
# 自分の種族値の中でcross_calcする
df_cross_calc = cross_calc_matrix(df_pok, ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])
df_pok_cross_calc = pd.concat([df_pok, df_cross_calc], axis=1)
Cols = df_pok_cross_calc.columns.drop(['#', 'Name', 'Type 1', 'Type 2'])

# 対戦表からキカイガクシューできる形(2ひきの種族値を並べる)にする
df_versus_matrix = make_matrix(df_tr, df_pok_cross_calc, Cols)
#df_versus_matrix_ts = make_matrix_test(df_ts, df_pok_cross_calc, Cols)
df_versus_matrix_ts = make_matrix_test(df_ts, df_pok_cross_calc, Cols)

# 対戦相手との四則演算を取るcolumns
relative_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed',
                'HP_add_Attack', 'HP_add_Defense', 'HP_add_Sp. Atk', 'HP_add_Sp. Def',
       'HP_add_Speed', 'Attack_add_Defense', 'Attack_add_Sp. Atk',
       'Attack_add_Sp. Def', 'Attack_add_Speed', 'Defense_add_Sp. Atk',
       'Defense_add_Sp. Def', 'Defense_add_Speed', 'Sp. Atk_add_Sp. Def',
       'Sp. Atk_add_Speed', 'Sp. Def_add_Speed', 'HP_sub_Attack',
       'HP_sub_Defense', 'HP_sub_Sp. Atk', 'HP_sub_Sp. Def', 'HP_sub_Speed',
       'Attack_sub_Defense', 'Attack_sub_Sp. Atk', 'Attack_sub_Sp. Def',
       'Attack_sub_Speed', 'Defense_sub_Sp. Atk', 'Defense_sub_Sp. Def',
       'Defense_sub_Speed', 'Sp. Atk_sub_Sp. Def', 'Sp. Atk_sub_Speed',
       'Sp. Def_sub_Speed', 'HP_multi_Attack', 'HP_multi_Defense',
       'HP_multi_Sp. Atk', 'HP_multi_Sp. Def', 'HP_multi_Speed',
       'Attack_multi_Defense', 'Attack_multi_Sp. Atk', 'Attack_multi_Sp. Def',
       'Attack_multi_Speed', 'Defense_multi_Sp. Atk', 'Defense_multi_Sp. Def',
       'Defense_multi_Speed', 'Sp. Atk_multi_Sp. Def', 'Sp. Atk_multi_Speed',
       'Sp. Def_multi_Speed', 'HP_div_Attack', 'HP_div_Defense',
       'HP_div_Sp. Atk', 'HP_div_Sp. Def', 'HP_div_Speed',
       'Attack_div_Defense', 'Attack_div_Sp. Atk', 'Attack_div_Sp. Def',
       'Attack_div_Speed', 'Defense_div_Sp. Atk', 'Defense_div_Sp. Def',
       'Defense_div_Speed', 'Sp. Atk_div_Sp. Def', 'Sp. Atk_div_Speed',
       'Sp. Def_div_Speed']
# 対戦相手との種族値(と種族値のcross_calcの)差をみる。
df_relative = relative_matrix(df_versus_matrix, relative_cols)
df_relative_ts = relative_matrix(df_versus_matrix_ts, relative_cols)
display(df_relative.head())

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 15360.11it/s]


,HP_pok_delta,Attack_pok_delta,Defense_pok_delta,Sp. Atk_pok_delta,Sp. Def_pok_delta,Speed_pok_delta,HP_add_Attack_pok_delta,HP_add_Defense_pok_delta,HP_add_Sp. Atk_pok_delta,HP_add_Sp. Def_pok_delta,HP_add_Speed_pok_delta,Attack_add_Defense_pok_delta,Attack_add_Sp. Atk_pok_delta,Attack_add_Sp. Def_pok_delta,Attack_add_Speed_pok_delta,Defense_add_Sp. Atk_pok_delta,Defense_add_Sp. Def_pok_delta,Defense_add_Speed_pok_delta,Sp. Atk_add_Sp. Def_pok_delta,Sp. Atk_add_Speed_pok_delta,Sp. Def_add_Speed_pok_delta,HP_sub_Attack_pok_delta,HP_sub_Defense_pok_delta,HP_sub_Sp. Atk_pok_delta,HP_sub_Sp. Def_pok_delta,HP_sub_Speed_pok_delta,Attack_sub_Defense_pok_delta,Attack_sub_Sp. Atk_pok_delta,Attack_sub_Sp. Def_pok_delta,Attack_sub_Speed_pok_delta,Defense_sub_Sp. Atk_pok_delta,Defense_sub_Sp. Def_pok_delta,Defense_sub_Speed_pok_delta,Sp. Atk_sub_Sp. Def_pok_delta,Sp. Atk_sub_Speed_pok_delta,Sp. Def_sub_Speed_pok_delta,HP_multi_Attack_pok_delta,HP_multi_Defense_pok_delta,HP_multi_Sp. Atk_pok_delta,HP_multi_Sp. Def_pok_delta,HP_multi_Speed_pok_delta,Attack_multi_Defense_pok_delta,Attack_multi_Sp. Atk_pok_delta,Attack_multi_Sp. Def_pok_delta,Attack_multi_Speed_pok_delta,Defense_multi_Sp. Atk_pok_delta,Defense_multi_Sp. Def_pok_delta,Defense_multi_Speed_pok_delta,Sp. Atk_multi_Sp. Def_pok_delta,Sp. Atk_multi_Speed_pok_delta,Sp. Def_multi_Speed_pok_delta,HP_div_Attack_pok_delta,HP_div_Defense_pok_delta,HP_div_Sp. Atk_pok_delta,HP_div_Sp. Def_pok_delta,HP_div_Speed_pok_delta,Attack_div_Defense_pok_delta,Attack_div_Sp. Atk_pok_delta,Attack_div_Sp. Def_pok_delta,Attack_div_Speed_pok_delta,Defense_div_Sp. Atk_pok_delta,Defense_div_Sp. Def_pok_delta,Defense_div_Speed_pok_delta,Sp. Atk_div_Sp. Def_pok_delta,Sp. Atk_div_Speed_pok_delta,Sp. Def_div_Speed_pok_delta
0,30.0,-25.0,110.0,50.0,0.0,-75.0,5.0,140.0,80.0,30.0,-45.0,85.0,25.0,-25.0,-100.0,160.0,110.0,35.0,50.0,-25.0,-75.0,55.0,-80.0,-20.0,30.0,105.0,-135.0,-75.0,-25.0,50.0,60.0,110.0,185.0,50.0,125.0,75.0,625.0,12550.0,7150.0,2400.0,-3975.0,6500.0,1750.0,-2000.0,-8250.0,17800.0,8800.0,-1950.0,4000.0,-4500.0,-6000.0,0.616667,-0.400794,-0.081731,0.375000,2.547619,-1.011905,-0.673077,-0.312500,1.547619,0.509615,1.375000,5.333333,0.625000,3.571429,1.904762
1,-10.0,-30.0,-65.0,0.0,-20.0,35.0,-40.0,-75.0,-10.0,-30.0,25.0,-95.0,-30.0,-50.0,5.0,-65.0,-85.0,-30.0,-20.0,35.0,15.0,20.0,55.0,-10.0,10.0,-45.0,35.0,-30.0,-10.0,-65.0,-65.0,-45.0,-100.0,20.0,-35.0,-55.0,-2600.0,-4575.0,-600.0,-1600.0,1425.0,-7225.0,-1800.0,-2800.0,775.0,-3900.0,-3950.0,-2025.0,-1200.0,2100.0,50.0,0.161943,0.921429,-0.166667,0.533333,-0.652941,0.907143,-0.500000,0.266667,-1.135294,-1.083333,-0.833333,-1.588235,0.800000,-0.494118,-0.647059
2,11.0,-13.0,-5.0,99.0,60.0,83.0,-2.0,6.0,110.0,71.0,94.0,-18.0,86.0,47.0,70.0,94.0,55.0,78.0,159.0,182.0,143.0,24.0,16.0,-88.0,-49.0,-72.0,-8.0,-112.0,-73.0,-96.0,-104.0,-65.0,-88.0,39.0,16.0,-23.0,-248.0,590.0,9339.0,5790.0,7828.0,-1595.0,6738.0,3930.0,5651.0,8760.0,5250.0,7345.0,10710.0,13182.0,8970.0,0.322712,0.169006,-1.961240,-1.655556,-2.357407,-0.094737,-2.275194,-2.033333,-2.733333,-2.468992,-2.166667,-2.966667,0.433333,-0.005556,-0.366667
3,25.0,35.0,-50.0,15.0,-70.0,40.0,60.0,-25.0,40.0,-45.0,65.0,-15.0,50.0,-35.0,75.0,-35.0,-120.0,-10.0,-55.0,55.0,-30.0,-10.0,75.0,10.0,95.0,-15.0,85.0,20.0,105.0,-5.0,-65.0,20.0,-90.0,85.0,-25.0,-110.0,4725.0,-1625.0,3025.0,-2950.0,5275.0,-1575.0,4025.0,-3150.0,6825.0,-2675.0,-9850.0,-1550.0,-4150.0,4525.0,-3250.0,-0.047619,0.974026,0.142857,1.200000,-0.060870,1.242424,0.235294,1.516667,-0.020290,-0.852941,0.225000,-0.921739,1.116667,-0.194203,-1.165217
4,25.0,-20.0,45.0,60.0,145.0,-35.0,5.0,70.0,85.0,170.0,-10.0,25.0,40.0,125.0,-55.0,105.0,190.0,10.0,205.0,25.0,110.0,45.0,-20.0,-35.0,-120.0,60.0,-65.0,-80.0,-165.0,15.0,-15.0,-100.0,80.0,-85.0,95.0,180.0,150.0,4975.0,5800.0,12975.0,-675.0,1150.0,2200.0,6150.0,-3450.0,7800.0,16975.0,325.0,17800.0,1600.0,5325.0,0.814286,-0.200000,-0.575000,-0.600000,0.952941,-0.772727,-1.250000,-1.022727,0.176471,-0.

Type変数について、交差項を作成

この相性同士だったら勝てるとかあるかなとおもったので

In [9]:
df_matrix = pd.concat([df_versus_matrix, df_relative], axis=1)
df_matrix_ts = pd.concat([df_versus_matrix_ts, df_relative_ts], axis=1)

poly = PolynomialFeatures(2, interaction_only=True)
type_cols = [i + '_pok1' for i in Type_names]
for i in Type_names:
    type_cols.append(i + '_pok2')
for i in attack_cols:
    type_cols.append(i + '_pok1')
    type_cols.append(i + '_pok2')
poly_tr = poly.fit_transform(df_matrix[type_cols])
poly_ts = poly.transform(df_matrix_ts[type_cols])
df_matrix = pd.concat([df_matrix, pd.DataFrame(poly_tr)], axis=1)
df_matrix_ts = pd.concat([df_matrix_ts, pd.DataFrame(poly_ts)], axis=1)
df_matrix = pd.concat([df_matrix, df_tr_maguni], axis=1)
df_matrix_ts = pd.concat([df_matrix_ts, df_ts_maguni], axis=1)

種族値の大きさについての基本統計量の変数を追加

In [10]:
type_pok1_cols = [i + '_pok1' for i in ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Speed', 'Sp. Def']]
type_pok2_cols = [i + '_pok2' for i in ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Speed', 'Sp. Def']]
df_matrix['max1'] = df_matrix[type_pok1_cols].max(axis=1)
df_matrix['min1'] = df_matrix[type_pok1_cols].min(axis=1)
df_matrix['std1'] = df_matrix[type_pok1_cols].std(axis=1)
df_matrix['mean1'] = df_matrix[type_pok1_cols].mean(axis=1)
df_matrix['max2'] = df_matrix[type_pok2_cols].max(axis=1)
df_matrix['min2'] = df_matrix[type_pok2_cols].min(axis=1)
df_matrix['std2'] = df_matrix[type_pok2_cols].std(axis=1)
df_matrix['mean2'] = df_matrix[type_pok2_cols].mean(axis=1)
df_matrix_ts['max1'] = df_matrix_ts[type_pok1_cols].max(axis=1)
df_matrix_ts['min1'] = df_matrix_ts[type_pok1_cols].min(axis=1)
df_matrix_ts['std1'] = df_matrix_ts[type_pok1_cols].std(axis=1)
df_matrix_ts['mean1'] = df_matrix_ts[type_pok1_cols].mean(axis=1)
df_matrix_ts['max2'] = df_matrix_ts[type_pok2_cols].max(axis=1)
df_matrix_ts['min2'] = df_matrix_ts[type_pok2_cols].min(axis=1)
df_matrix_ts['std2'] = df_matrix_ts[type_pok2_cols].std(axis=1)
df_matrix_ts['mean2'] = df_matrix_ts[type_pok2_cols].mean(axis=1)
display(df_matrix[['max1', 'min1', 'std1', 'mean1']].head())

,max1,min1,std1,mean1
0,180.0,30.0,51.445764,98.333333
1,85.0,30.0,20.248457,55.000000
2,129.0,72.0,19.510681,96.666667
3,115.0,50.0,26.028830,82.500000
4,200.0,50.0,55.377492,96.666667


対戦する個別のポケモンやそのTypeをcategorical変数と考えて、likelihood encoding

dataframeをfor文で1行ずつみるというかなり頭の悪いことをしています。。。（applyしよう）

In [11]:
df_tr = pd.read_csv('../input/train.csv', index_col='id')
df_ts = pd.read_csv('../input/test.csv', index_col='id')

df_matrix['F_pok'] = df_tr['First_pokemon']
df_matrix['S_pok'] = df_tr['Second_pokemon']
df_matrix_ts['F_pok'] = df_ts['First_pokemon']
df_matrix_ts['S_pok'] = df_ts['Second_pokemon']
df_matrix['T1_pok1'] = np.nan
df_matrix['T2_pok1'] = np.nan
df_matrix['T1_pok2'] = np.nan
df_matrix['T2_pok2'] = np.nan
df_matrix_ts['T1_pok1'] = np.nan
df_matrix_ts['T2_pok1'] = np.nan
df_matrix_ts['T1_pok2'] = np.nan
df_matrix_ts['T2_pok2'] = np.nan
for i in tqdm(range(len(df_matrix))):
    pok1 = df_matrix.loc[i, 'F_pok']
    pok2 = df_matrix.loc[i, 'S_pok']
    df_matrix.loc[i, 'T1_pok1'] = df_pok.loc[pok1-1, 'Type 1']
    df_matrix.loc[i, 'T2_pok1'] = df_pok.loc[pok1-1, 'Type 2']
    df_matrix.loc[i, 'T1_pok2'] = df_pok.loc[pok2-1, 'Type 1']
    df_matrix.loc[i, 'T2_pok2'] = df_pok.loc[pok2-1, 'Type 2']
    
for i in tqdm(range(len(df_matrix_ts))):
    pok1 = df_matrix_ts.loc[i, 'F_pok']
    pok2 = df_matrix_ts.loc[i, 'S_pok']
    df_matrix_ts.loc[i, 'T1_pok1'] = df_pok.loc[pok1-1, 'Type 1']
    df_matrix_ts.loc[i, 'T2_pok1'] = df_pok.loc[pok1-1, 'Type 2']
    df_matrix_ts.loc[i, 'T1_pok2'] = df_pok.loc[pok2-1, 'Type 1']
    df_matrix_ts.loc[i, 'T2_pok2'] = df_pok.loc[pok2-1, 'Type 2']

df_matrix = df_matrix.fillna(-9999)
df_matrix_ts = df_matrix_ts.fillna(-9999)

tae_cols = ['t1_pok1_te', 't2_pok1_te', 't1_pok2_te', 't2_pok2_te', 'pok1_te', 'pok2_te', 't1_pok1_co', 't2_pok1_co', 't1_pok2_co', 't2_pok2_co', 'pok1_co', 'pok2_co']

df_matrix['t1_pok1_te'] = df_matrix['T1_pok1'].map(df_matrix.groupby('T1_pok1')['target'].mean())
df_matrix['t2_pok1_te'] = df_matrix['T2_pok1'].map(df_matrix.groupby('T2_pok1')['target'].mean())
df_matrix['t1_pok2_te'] = df_matrix['T1_pok2'].map(df_matrix.groupby('T1_pok2')['target'].mean())
df_matrix['t2_pok2_te'] = df_matrix['T2_pok2'].map(df_matrix.groupby('T2_pok2')['target'].mean())
df_matrix_ts['t1_pok1_te'] = df_matrix_ts['T1_pok1'].map(df_matrix.groupby('T1_pok1')['target'].mean())
df_matrix_ts['t2_pok1_te'] = df_matrix_ts['T2_pok1'].map(df_matrix.groupby('T2_pok1')['target'].mean())
df_matrix_ts['t1_pok2_te'] = df_matrix_ts['T1_pok2'].map(df_matrix.groupby('T1_pok2')['target'].mean())
df_matrix_ts['t2_pok2_te'] = df_matrix_ts['T2_pok2'].map(df_matrix.groupby('T2_pok2')['target'].mean())
df_matrix['pok1_te'] = df_matrix['F_pok'].map(df_matrix.groupby('F_pok')['target'].mean())
df_matrix['pok2_te'] = df_matrix['S_pok'].map(df_matrix.groupby('S_pok')['target'].mean())
df_matrix_ts['pok1_te'] = df_matrix_ts['F_pok'].map(df_matrix.groupby('F_pok')['target'].mean())
df_matrix_ts['pok2_te'] = df_matrix_ts['S_pok'].map(df_matrix.groupby('S_pok')['target'].mean())
df_matrix['t1_pok1_co'] = df_matrix['T1_pok1'].map(df_matrix.groupby('T1_pok1')['target'].count())
df_matrix['t2_pok1_co'] = df_matrix['T2_pok1'].map(df_matrix.groupby('T2_pok1')['target'].count())
df_matrix['t1_pok2_co'] = df_matrix['T1_pok2'].map(df_matrix.groupby('T1_pok2')['target'].count())
df_matrix['t2_pok2_co'] = df_matrix['T2_pok2'].map(df_matrix.groupby('T2_pok2')['target'].count())
df_matrix_ts['t1_pok1_co'] = df_matrix_ts['T1_pok1'].map(df_matrix.groupby('T1_pok1')['target'].count())
df_matrix_ts['t2_pok1_co'] = df_matrix_ts['T2_pok1'].map(df_matrix.groupby('T2_pok1')['target'].count())
df_matrix_ts['t1_pok2_co'] = df_matrix_ts['T1_pok2'].map(df_matrix.groupby('T1_pok2')['target'].count())
df_matrix_ts['t2_pok2_co'] = df_matrix_ts['T2_pok2'].map(df_matrix.groupby('T2_pok2')['target'].count())
df_matrix['pok1_co'] = df_matrix['F_pok'].map(df_matrix.groupby('F_pok')['target'].count())
df_matrix['pok2_co'] = df_matrix['S_pok'].map(df_matrix.groupby('S_pok')['target'].count())
df_matrix_ts['pok1_co'] = df_matrix_ts['F_pok'].map(df_matrix.groupby('F_pok')['target'].count())
df_matrix_ts['pok2_co'] = df_matrix_ts['S_pok'].map(df_matrix.groupby('S_pok')['target'].count())
df_matrix, df_matrix_ts = WoE(df_matrix, df_matrix_ts, 'F_pok', 'target')
df_matrix, df_matrix_ts = WoE(df_matrix, df_matrix_ts, 'S_pok', 'target')
df_matrix, df_matrix_ts = WoE(df_matrix, df_matrix_ts, 'T1_pok1', 'target')
df_matrix, df_matrix_ts = WoE(df_matrix, df_matrix_ts, 'T2_pok1', 'target')
df_matrix, df_matrix_ts = WoE(df_matrix, df_matrix_ts, 'T1_pok2', 'target')
df_matrix, df_matrix_ts = WoE(df_matrix, df_matrix_ts, 'T2_pok2', 'target')
cross_tae = cross_calc_matrix(df_matrix, tae_cols)
cross_tae_ts = cross_calc_matrix(df_matrix_ts, tae_cols)
df_matrix = pd.concat([df_matrix, cross_tae], axis=1)
df_matrix_ts = pd.concat([df_matrix_ts, cross_tae_ts], axis=1)


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 94.05it/s]
C:\Users\sugawara\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
C:\Users\sugawara\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


要らない変数を削除

In [12]:
del  df_pok_cross_calc
del  df_relative
del  df_relative_ts
del  df_tr
del  df_tr_maguni
del  df_ts
del  df_ts_maguni
del  df_type_ohe
del  df_versus_matrix
del  df_versus_matrix_ts
del  poly_tr
del  poly_ts
del  type1_1st_list
del  type1_2nd_list
del  type2_1st_list
del  type2_2nd_list
del  df_mx
del  attack_1st_magni_list
del  attack_2nd_magni_list
del  attack_matrix
del  cross_maguni_tr
del  cross_maguni_ts
del  cross_tae
del  cross_tae_ts
del  df_cross_calc
del  df_generation_ohe

Lightgbmで予測

In [13]:
%%time
X_cols = df_matrix.columns.drop(['target', 'T1_pok1', 'T2_pok1', 'T1_pok2', 'T2_pok2'])
#kf = KFold(n_splits=5,shuffle=True, random_state=71)
clf = LGBMClassifier(learning_rate=0.03, n_estimators=600, num_leaves = 45, random_state=71, reg_lambda=0.6, reg_alpha=0.4)
clf.fit(df_matrix[X_cols], df_matrix['target'])
pred = clf.predict_proba(df_matrix_ts[X_cols])[:, 1]
sub = pd.DataFrame()
sub['id'] = np.linspace(0, 9999, 10000)
sub['probability'] = pred
sub.to_csv('../output/1st_place_submit.csv', index=False)

C:\Users\sugawara\Anaconda3\lib\site-packages\lightgbm\basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
C:\Users\sugawara\Anaconda3\lib\site-packages\lightgbm\basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


Wall time: 10min 2s
